In [6]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00


In [10]:
pip install tensorflow torch datasets


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from datasets import load_dataset

In [13]:
# Load dataset
dataset = load_dataset("carblacac/twitter-sentiment-analysis")
def preprocess_data(data):
  tweets = data['text']
  labels = data['sentiment']

In [14]:
def preprocess_data(example):
  return {'text': example['text'],'label': example['feeling']}

In [15]:
print(dataset.keys())

dict_keys(['train', 'validation', 'test'])


In [16]:
print(dataset['train'][0])

{'text': '@fa6ami86 so happy that salman won.  btw the 14sec clip is truely a teaser', 'feeling': 0}


In [21]:
train_data = dataset['train'].map(preprocess_data)
test_data = dataset['test'].map(preprocess_data)

In [22]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token = '<OOV>')
tokenizer.fit_on_texts(train_data['text'])
vocab_size = len(tokenizer.word_index)+1

In [23]:
 train_sequences = tokenizer.texts_to_sequences(train_data['text'])
 test_sequences = tokenizer.texts_to_sequences(test_data['text'])

In [26]:
max_length = 50
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen= max_length, padding = 'post', truncating='post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen= max_length, padding = 'post', truncating='post')

In [28]:
embedding_dim = 16

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    Bidirectional (LSTM(64, return_sequences=True)),
    Bidirectional (LSTM(32)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [29]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [30]:
import numpy as np

In [31]:
train_labels = np.array(train_data['label'])
test_labels = np.array(train_data['label'])

In [32]:
num_epochs = 5
model.fit(train_padded, train_labels, epochs = num_epochs, validation_data =(test_padded,test_labels))

Epoch 1/5
3750/3750 [==============================] - 582s 153ms/step - loss: 0.4811 - accuracy: 0.7667 - val_loss: 0.3565 - val_accuracy: 0.8530
Epoch 2/5
3750/3750 [==============================] - 495s 132ms/step - loss: 0.3246 - accuracy: 0.8612 - val_loss: 0.2227 - val_accuracy: 0.9162
Epoch 3/5
3750/3750 [==============================] - 497s 133ms/step - loss: 0.2228 - accuracy: 0.9096 - val_loss: 0.1452 - val_accuracy: 0.9468
Epoch 4/5
3750/3750 [==============================] - 497s 132ms/step - loss: 0.1591 - accuracy: 0.9364 - val_loss: 0.1051 - val_accuracy: 0.9607
Epoch 5/5
3750/3750 [==============================] - 497s 133ms/step - loss: 0.1193 - accuracy: 0.9528 - val_loss: 0.0752 - val_accuracy: 0.9716


In [35]:
loss, accuracy = model.evaluate(test_padded, test_labels)
print(f'Loss:{loss},Accuracy: {accuracy}')


3750/3750 [==============================] - 91s 24ms/step - loss: 0.0752 - accuracy: 0.9716
Loss:0.07517445832490921,Accuracy: 0.9715554714202881


In [36]:
sample_text = ["I love this product", "this is the worst experuence ever."]
sample_sequences = tokenizer.texts_to_sequences(sample_text)
sample_padded = tf.keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen = max_length, padding = 'post', truncating = 'post')


In [38]:
predictions = model.predict(sample_padded)
for i, text in enumerate(sample_text):
  sentiment = 'Positive' if predictions[i]>0.5 else 'Negative'
  print(f'Text: {text}, Predicted Sentiment: {sentiment}')

1/1 [==============================] - 0s 30ms/step
Text: I love this product, Predicted Sentiment: Positive
Text: this is the worst experuence ever., Predicted Sentiment: Negative
